## 📖 Read Rules and Facts from Files

In [137]:
def parse_facts(facts_path='facts.txt'):
  facts = []
  with open(facts_path) as file:
    for line in file:
      facts.append(line.strip())
  return facts

In [138]:
def parse_rules(rules_path='rules.txt'):
  rules = {}
  parsed_rules = []
  
  with open(rules_path) as file:
    for i, line in enumerate(file):
      rules[f'R{i}'] = line.strip()
      
      # parts[0] -> premises
      # part[1] -> conclusion
      parts = rules[f'R{i}'].strip().split('THEN')
      
      # Considering ANDs
      raw_premises = parts[0].replace('IF', '').strip().split('AND')
      premises = []

      # considering ORs in ANDs
      for p in raw_premises:
        # [['A','B'], ['C']] = ('A' OR 'B') AND 'C'
        ors = [condition.strip() for condition in p.strip().split('OR')]
        premises.append(ors)

      conclusion = parts[1].strip()

      parsed_rules.append({
              'id': f'R{i}',
              'premises': premises,
              'conclusion': conclusion
          })
      
  return rules, parsed_rules

In [139]:
rules, parsed_rules = parse_rules()
original_facts = parse_facts()
goal = 'citrus_fruit'

# Make a copy of original use for appropriate use in Backward chaining
facts = original_facts[:]

In [140]:
original_facts

['seeds = 0', 'diameter = 7', 'skin_smell', 'color is orange']

In [141]:
rules

{'R0': 'IF shape is long AND color is yellow THEN fruit is banana',
 'R1': 'IF shape is round AND color is red AND size is medium THEN fruit is apple',
 'R2': 'IF shape is round AND color is red AND size is small THEN fruit is cherry',
 'R3': 'IF skin_smell THEN perfumed',
 'R4': 'IF fruit is lemon OR fruit is orange OR fruit is pomelo OR fruit is grapefruit THEN citrus_fruit',
 'R5': 'IF size is medium AND color is yellow AND perfumed THEN fruit is lemon',
 'R6': 'IF size is medium AND color is green THEN fruit is kiwi',
 'R7': 'IF size is big AND perfumed AND color is orange AND citrus_fruit THEN fruit is grapefruit',
 'R8': 'IF perfumed AND color is orange AND size is medium THEN fruit is orange',
 'R9': 'IF perfumed AND color is red AND size is small AND seeds = 0 THEN fruit is strawberry',
 'R10': 'IF diameter < 2 THEN size is small',
 'R11': 'IF diameter > 10 THEN size is big',
 'R12': 'IF diameter > 2 AND diameter < 10 THEN size is medium'}

In [142]:
parsed_rules

[{'id': 'R0',
  'premises': [['shape is long'], ['color is yellow']],
  'conclusion': 'fruit is banana'},
 {'id': 'R1',
  'premises': [['shape is round'], ['color is red'], ['size is medium']],
  'conclusion': 'fruit is apple'},
 {'id': 'R2',
  'premises': [['shape is round'], ['color is red'], ['size is small']],
  'conclusion': 'fruit is cherry'},
 {'id': 'R3', 'premises': [['skin_smell']], 'conclusion': 'perfumed'},
 {'id': 'R4',
  'premises': [['fruit is lemon',
    'fruit is orange',
    'fruit is pomelo',
    'fruit is grapefruit']],
  'conclusion': 'citrus_fruit'},
 {'id': 'R5',
  'premises': [['size is medium'], ['color is yellow'], ['perfumed']],
  'conclusion': 'fruit is lemon'},
 {'id': 'R6',
  'premises': [['size is medium'], ['color is green']],
  'conclusion': 'fruit is kiwi'},
 {'id': 'R7',
  'premises': [['size is big'],
   ['perfumed'],
   ['color is orange'],
   ['citrus_fruit']],
  'conclusion': 'fruit is grapefruit'},
 {'id': 'R8',
  'premises': [['perfumed'], ['col

## Implement Forward Chaining from scratch

In [143]:
def Forward_Chaining(rules, parsed_rules, original_facts, goal):
    facts = original_facts[:]
    Fired_rules = []
    cycle = 1

    # For assigned variables -> EX: 'diameter = 7'
    fact_vars = {}

    # Add assignments to fact_vars
    for fact in facts:
        if '=' in fact and '==' not in fact:
            exec(fact, {}, fact_vars)

    while goal not in facts and len(Fired_rules) < len(rules):
        print(f'Cycle {cycle}:')
        rule_fired_this_cycle = False

        for rule in parsed_rules:
            id = rule['id']
            premises = rule['premises']
            conclusion = rule['conclusion']

            # Skip if rule is already fired
            if id in Fired_rules:
                continue

            rule_match = True
            for ANDs in premises:
                match = False
                for ORs in ANDs:
                    # Check if it's a known fact
                    if ORs in facts:
                        match = True
                        break
                    try:
                        # Try evaluating the condition
                        if eval(ORs, {}, fact_vars):
                            match = True
                            break
                    except:
                        pass

                # If the OR group has none matched; entire rule fails
                if not match:
                    rule_match = False
                    break

            if rule_match:
                Fired_rules.append(id)
                facts.append(conclusion)

                # IF there are conclusions to be added to fact_vars
                try:
                    if '=' in conclusion and '==' not in conclusion:
                        exec(conclusion, {}, fact_vars)
                except:
                    pass

                # Add the conclusion to facts and print the updated facts
                print(f'- Fire {id}: {rules[id]}\n'+
                    f'- Add \'{conclusion}\' to facts.\n'+
                    f'- Facts = {facts}\n')
                
                rule_fired_this_cycle = True

                # Fire only one rule per cycle
                break

        # Break if no rule is fired during the cycle
        if not rule_fired_this_cycle:
            print('- No rules to fire.\n')
            break

        if goal not in facts:
            print('=> Goal not reached; Continue.\n' +
                  '-' * 50)

        cycle += 1
    print('😊 Goal was reached successfully.') if goal in facts else print('😞 Failed to reach goal.\n')
    return

In [144]:
Forward_Chaining(rules, parsed_rules, original_facts, goal)

Cycle 1:
- Fire R3: IF skin_smell THEN perfumed
- Add 'perfumed' to facts.
- Facts = ['seeds = 0', 'diameter = 7', 'skin_smell', 'color is orange', 'perfumed']

=> Goal not reached; Continue.
--------------------------------------------------
Cycle 2:
- Fire R12: IF diameter > 2 AND diameter < 10 THEN size is medium
- Add 'size is medium' to facts.
- Facts = ['seeds = 0', 'diameter = 7', 'skin_smell', 'color is orange', 'perfumed', 'size is medium']

=> Goal not reached; Continue.
--------------------------------------------------
Cycle 3:
- Fire R8: IF perfumed AND color is orange AND size is medium THEN fruit is orange
- Add 'fruit is orange' to facts.
- Facts = ['seeds = 0', 'diameter = 7', 'skin_smell', 'color is orange', 'perfumed', 'size is medium', 'fruit is orange']

=> Goal not reached; Continue.
--------------------------------------------------
Cycle 4:
- Fire R4: IF fruit is lemon OR fruit is orange OR fruit is pomelo OR fruit is grapefruit THEN citrus_fruit
- Add 'citrus_f

## Implement Backward Chaining from scratch

In [ ]:
def Backward_Chaining(rules, parsed_rules, facts, goal, indent=0):
    # For assigned variables -> EX: 'diameter = 7'
    fact_vars = {}
    for fact in facts:
        if '=' in fact and '==' not in fact:
            exec(fact, {}, fact_vars)

    # If the goal is in facts or evaluates to True
    try:
        if goal in facts or eval(goal, {}, fact_vars):
            print(f'{' ' * indent}✔️ \'{goal}\' is in Facts')
            return True
    except:
        pass

    print(f'{' ' * indent}* Matching \'{goal}\'')
    
    for rule in parsed_rules:
        if rule['conclusion'] == goal:
            id = rule['id']
            premises = rule['premises']
            print(f'{' ' * indent}- {id}: {rules[id]}')

            rule_match = True
            for ANDs in premises:
                match = False
                for ORs in ANDs:
                    # Check if it's a known fact
                    if Backward_Chaining(rules, parsed_rules, facts, ORs, indent + 4):
                        match = True
                        break
                
                # If the OR group has none matched; entire rule fails
                if not match:
                    rule_match = False
                    break

            if rule_match:
                if goal not in facts:
                    facts.append(goal)

                    # IF there are conclusions to be added to fact_vars
                    if '=' in goal and '==' not in goal:
                        try:
                            exec(goal, {}, fact_vars)
                        except:
                            pass

                    # Add the goal to facts and print the updated facts
                    print(f'{' ' * indent}✅ {id} is True.\n' +
                        f'{' ' * indent}- Add \'{goal}\' to Facts.\n' +
                        f'{' ' * indent}- Facts = {facts}\n')
                    return True
            else:
                print(f'{' ' * indent}❌ {id} is False.')

    print(f'{' ' * indent}⚠️ Could not prove: \'{goal}\'\n')
    return False

In [146]:
print(f'=> Starting with facts = {original_facts} to prove goal: \'{goal}\'\n')
print('😊 Goal was reached successfully.\n') if Backward_Chaining(rules, parsed_rules, facts, goal) else print('😞 Failed to reach goal.\n')

=> Starting with facts = ['seeds = 0', 'diameter = 7', 'skin_smell', 'color is orange'] to prove goal: 'citrus_fruit'

* Matching 'citrus_fruit'
- R4: IF fruit is lemon OR fruit is orange OR fruit is pomelo OR fruit is grapefruit THEN citrus_fruit
    * Matching 'fruit is lemon'
    - R5: IF size is medium AND color is yellow AND perfumed THEN fruit is lemon
        * Matching 'size is medium'
        - R12: IF diameter > 2 AND diameter < 10 THEN size is medium
            ✔️ 'diameter > 2' is in Facts
            ✔️ 'diameter < 10' is in Facts
        ✅ R12 is True.
        - Add 'size is medium' to Facts.
        - Facts = ['seeds = 0', 'diameter = 7', 'skin_smell', 'color is orange', 'size is medium']

        * Matching 'color is yellow'
        ⚠️ Could not prove: 'color is yellow'

    ❌ R5 is False.
    ⚠️ Could not prove: 'fruit is lemon'

    * Matching 'fruit is orange'
    - R8: IF perfumed AND color is orange AND size is medium THEN fruit is orange
        * Matching 'perfu